In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

import pandas as pd
import random
from dataclasses import dataclass
import numpy as np
import torch
# import seaborn as sns
import transformers
import json
import glob
from tqdm import tqdm
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
from torchmetrics import MetricCollection
from torchmetrics.classification import Accuracy, AUROC, F1Score, Precision, Recall
from itertools import chain

# seed torch operations
SEED = 13
torch.manual_seed(SEED)

/home/tbrow51/.conda/envs/transformers/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"using {device} device...")

using cuda device...


In [3]:
file_list = glob.glob("/opt/localdata/Data/bea/nlp/bmi550/project/chronic_pain_model_data/*.csv")
file_list

['/opt/localdata/Data/bea/nlp/bmi550/project/chronic_pain_model_data/dev.csv',
 '/opt/localdata/Data/bea/nlp/bmi550/project/chronic_pain_model_data/test.csv',
 '/opt/localdata/Data/bea/nlp/bmi550/project/chronic_pain_model_data/train.csv']

In [4]:
df_list = [pd.read_csv(f) for f in file_list]
df = pd.concat(df_list, axis=0).reset_index()
df

,index,tweet_id,text,label
0,0,1365360731055673345,new publication. chronic pain assessment is un...,0
1,1,1365571852245078018,today is a bad day for me. chronic pain is a b...,1
2,2,1365710497903960066,<hashtag> call for submissions\n\nif you’re a...,0
3,3,1365783013674606598,chronic pain does not <allcaps> have to contro...,0
4,4,1366344621786341381,call for participants for <user> phd research:...,0
...,...,...,...,...
3294,2105,1478736266682408961,"chronic pain and the self pity, depression tra...",0
3295,2106,1478743181357469697,pinpointing pain is not always easy as our bod...,0
3296,2107,1478787131451625477,“it is just amazing how chronic pain can paral...,0
3297,2108,1478801225399238656,anyone i know with autoimmune / chronic pain i...,1


In [5]:
df.drop_duplicates()

,index,tweet_id,text,label
0,0,1365360731055673345,new publication. chronic pain assessment is un...,0
1,1,1365571852245078018,today is a bad day for me. chronic pain is a b...,1
2,2,1365710497903960066,<hashtag> call for submissions\n\nif you’re a...,0
3,3,1365783013674606598,chronic pain does not <allcaps> have to contro...,0
4,4,1366344621786341381,call for participants for <user> phd research:...,0
...,...,...,...,...
3294,2105,1478736266682408961,"chronic pain and the self pity, depression tra...",0
3295,2106,1478743181357469697,pinpointing pain is not always easy as our bod...,0
3296,2107,1478787131451625477,“it is just amazing how chronic pain can paral...,0
3297,2108,1478801225399238656,anyone i know with autoimmune / chronic pain i...,1


In [6]:
df.label.value_counts(dropna=False)

label
0    2765
1     534
Name: count, dtype: int64

In [7]:
@dataclass
class FrameParams:
    df: pd.DataFrame
    class_name: str
    class_val: float


# function to get the set of unique patient ids in the dataframe
# then split based on the train/val/test proportion
def split_ids(id_col, test_prop, validation, seed):
    # get set of unique ids and convert to a list
    id_list = list(set(id_col))

    # shuffle id list
    random.Random(seed).shuffle(id_list)

    # get split lengths
    id_list_len = len(id_list)

    # get the length of indexes to add to the train/test sets
    train_prop = 1.0 - (2 * test_prop)
    train_len = int(train_prop * id_list_len)
    test_len = int(test_prop * id_list_len)

    # index set ids
    if validation:
        train_ids = id_list[:train_len]
        val_ids = id_list[train_len:train_len+test_len]

    else:
        train_ids = id_list[:train_len+test_len]
        val_ids = None

    test_ids = id_list[train_len+test_len:]

    print('total ids:', id_list_len)

    print('train ids: {}, prop: {:.3f}'.format(
        len(train_ids),
        len(train_ids) / id_list_len
    ))

    if validation:
        print('val ids: {}, prop: {:.3f}'.format(
            len(val_ids),
            len(val_ids) / id_list_len
        ))

    print('test ids: {}, prop: {:.3f}\n'.format(
        len(test_ids),
        len(test_ids) / id_list_len
    ))

    return train_ids, val_ids, test_ids

# function to index pos/neg dataframes by set patient ids and merge them
def index_dataframes(df_obj_list, ids, id_var):
    # zip pos/neg dataframes and ids
    components = zip([df_obj.df for df_obj in df_obj_list], ids)

    # index dataframes by ids for pos/neg
    df_list = [df[df[id_var].isin(ids)] for df, ids in components]

    # merge pos/neg dataframes
    out_df = pd.concat(df_list, axis=0)
    return out_df

# function to split a positive and negative dataframe into train/val/test
# then merge positive and negative for each
def split_n_dataframes(df_list, id_var: str = 'tweet_id',
                       test_prop: float = 0.2, seed: int = 13,
                       validation: bool = True, label_col: str = 'label'):
    # add label columns to dataframes
    for df_obj in df_list:
        df_obj.df.loc[:, 'class_label'] = df_obj.class_val

    # get empty list to put dataframe set IDs
    df_ids = []

    # get ids for each split dataframe
    for df_obj in df_list:
        train_ids, val_ids, test_ids = split_ids(
            df_obj.df[id_var],
            test_prop,
            validation,
            seed
        )
        df_ids.append([train_ids, val_ids, test_ids])

    # transpose list to get sublists of all train set IDs, val sets IDs, etc.
    trans_df_ids = [i for i in zip(*df_ids)]

    # prepare lists for indexing
    train_ids = trans_df_ids[0]
    val_ids = trans_df_ids[1]
    test_ids = trans_df_ids[2]

    # index split dataframes
    train_df = index_dataframes(df_list, train_ids, id_var)
    test_df = index_dataframes(df_list, test_ids, id_var)
    if validation:
        val_df = index_dataframes(df_list, val_ids, id_var)

    # shuffle dataframes
    train_df = train_df.sample(frac=1, random_state=seed).reset_index()
    test_df = test_df.sample(frac=1, random_state=seed).reset_index()
    if validation:
        val_df = val_df.sample(frac=1, random_state=seed).reset_index()
    else:
        val_df = None

    return train_df, val_df, test_df

In [8]:
# define constants
TEST_SIZE = 0.2
VALIDATION = True
SEED = 13

# split on label first
pos_df = df[df['label'] == 1]
neg_df = df[df['label'] == 0]

df_list = [FrameParams(pos_df, 'pos', 1.0), FrameParams(neg_df, 'neg', 0.0)]

# split each label df by username, then combine them into a single
# dataframe for each split
train_df, val_df, test_df = split_n_dataframes(
    df_list,
    test_prop=TEST_SIZE,
    validation=VALIDATION,
    seed=SEED
)

print('train size:', len(train_df))
print('val size:', len(val_df))
print('test size:', len(test_df))

total ids: 534
train ids: 320, prop: 0.599
val ids: 106, prop: 0.199
test ids: 108, prop: 0.202

total ids: 2765
train ids: 1659, prop: 0.600
val ids: 553, prop: 0.200
test ids: 553, prop: 0.200

train size: 1979
val size: 659
test size: 661


/tmp/ipykernel_1148482/1235383812.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_obj.df.loc[:, 'class_label'] = df_obj.class_val
/tmp/ipykernel_1148482/1235383812.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_obj.df.loc[:, 'class_label'] = df_obj.class_val


In [10]:
print(f'\ntrain distribution:\n{train_df.label.value_counts(dropna=False, normalize=True)}')
print(f'\nval distribution:\n{val_df.label.value_counts(dropna=False, normalize=True)}')
print(f'\ntest distribution:\n{test_df.label.value_counts(dropna=False, normalize=True)}')


train distribution:
label
0    0.838302
1    0.161698
Name: proportion, dtype: float64

val distribution:
label
0    0.83915
1    0.16085
Name: proportion, dtype: float64

test distribution:
label
0    0.836611
1    0.163389
Name: proportion, dtype: float64


In [11]:
print(f'\ntrain distribution:\n{train_df.label.value_counts(dropna=False, normalize=False)}')
print(f'\nval distribution:\n{val_df.label.value_counts(dropna=False, normalize=False)}')
print(f'\ntest distribution:\n{test_df.label.value_counts(dropna=False, normalize=False)}')


train distribution:
label
0    1659
1     320
Name: count, dtype: int64

val distribution:
label
0    553
1    106
Name: count, dtype: int64

test distribution:
label
0    553
1    108
Name: count, dtype: int64


In [10]:
# load roberta base as a tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

In [11]:
class TweetDataset(Dataset):
    """
    class is very closely based on the huggingface tutorial implementation
    """
    def __init__(self, dataframe, tokenizer, max_len, id_col: str = 'tweet_id',
                 text_col: str = 'text', target_col: str = 'class_label'):
        self.tokenizer = tokenizer
        # self.data = dataframe
        self.tweet_id_list = list(dataframe[id_col])
        self.text_list = list(dataframe[text_col])
        self.label_list = list(dataframe[target_col])
        self.max_len = max_len

    def __len__(self):
        # get length of dataset (required for dataloader)
        return len(self.text_list)

    def __getitem__(self, idx):
        # extract text
        text = str(self.text_list[idx])

        # extract label
        label = self.label_list[idx]

        # tokenize text
        encoded_text = self.tokenizer.encode_plus(
            text,
            # add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True
        )

        # unpack encoded text
        ids = encoded_text['input_ids']
        attention_mask = encoded_text['attention_mask']
        token_type_ids = encoded_text["token_type_ids"]

        # wrap outputs in dict
        out_dict = {
            'tweet_id_list': self.tweet_id_list,
            'id_tensor': torch.tensor(ids, dtype=torch.long),
            'mask_tensor': torch.tensor(attention_mask, dtype=torch.long),
            'token_type_tensor': torch.tensor(token_type_ids, dtype=torch.long),
            'label_tensor': torch.tensor(label, dtype=torch.float)
        }

        return out_dict

In [12]:
MAX_LEN = 256

# load dataframes into dataset objects
train_ds = TweetDataset(train_df, tokenizer, MAX_LEN)
val_ds = TweetDataset(val_df, tokenizer, MAX_LEN)
test_ds = TweetDataset(test_df, tokenizer, MAX_LEN)

In [13]:
def get_dataloader(dataset, batch_size, shuffle: bool = True,
                   pin_memory: bool = True, num_workers: int = 0,
                   prefetch_factor: int or None = None):
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        pin_memory=pin_memory,
        num_workers=num_workers,
        prefetch_factor=prefetch_factor
    )
    return dataloader

BATCH_SIZE = 128

# load datasets into loaders
train_loader = get_dataloader(train_ds, BATCH_SIZE)
val_loader = get_dataloader(val_ds, BATCH_SIZE)
test_loader = get_dataloader(test_ds, BATCH_SIZE)

In [14]:
class CustomRoberta(torch.nn.Module):
    """
    model subclass to define the RoBERTa architecture, also closely based on
    the huggingface tutorial implementation
    """
    def __init__(self, drop_percent, num_classes, pt_model_name: str = 'roberta-base'):
        super().__init__()
        self.base_model = RobertaModel.from_pretrained(pt_model_name)
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(drop_percent)
        self.classifier = torch.nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask, token_type_ids):
        # get outputs from base model
        base_outputs = self.base_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        # extract hidden state from roberta base outputs
        hidden_state = base_outputs[0]
        x = hidden_state[:, 0]

        # define the linear layer preceding the classifier
        # and apply ReLU activation to its outputs
        x = self.pre_classifier(x)
        x = torch.nn.ReLU()(x)

        # define the dropout layer and classifier
        # and apply Sigmoid activation to its outputs
        x = self.dropout(x)
        x = self.classifier(x)
        outputs = torch.nn.Sigmoid()(x)
        return outputs

In [15]:
# build the model and send it to the gpu
model = CustomRoberta(0.3, 1)
model.to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


CustomRoberta(
  (base_model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): La

In [16]:
def train_model(model, loader_dict, metric_collection, 
                criterion, optimizer, save_dir: str or None = None, 
                num_epochs: int = 25, monitor_metric: str = 'val_loss'):
    if save_dir is not None:
        # if save dir doesn't exist, make it
        Path(save_dir).mkdir(parents=True, exist_ok=True)
        model_save_path = os.path.join(save_dir, 'best_model_params.pth')
    
    # save base weights
    torch.save(model.state_dict(), model_save_path)

    # initialize the best metric based on what the monitor metric is
    # (and if it should be maximized or minimized)
    if monitor_metric.split('_')[-1] == 'loss':
        best_metric = np.inf
    else:
        best_metric = -np.inf

    # iterate over epochs
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch} {'-' * 40}")

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            # running_size = 0

            # select current data loader
            phase_loader = loader_dict[phase]
            phase_size = len(phase_loader)

            # iterate over data in current phase loader
            with tqdm(phase_loader, unit="batch", total=phase_size) as epoch_iter:
                for batch, data in enumerate(epoch_iter):
                    # unpack data dict
                    id_tensor = data['id_tensor'].to(device)
                    mask_tensor = data['mask_tensor'].to(device)
                    token_type_tensor = data['token_type_tensor'].to(device)
                    label_tensor = data['label_tensor'].to(device)
                    
                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(
                            id_tensor,
                            mask_tensor,
                            token_type_tensor
                        )
                        preds = torch.squeeze(outputs)
                        loss = criterion(preds, label_tensor)

                        # update running loss
                        running_loss += loss.item() #* label_tensor.size(0)
                        # running_size += label_tensor.size(0)

                        # update metric collection
                        metric_collection.update(preds, label_tensor)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # update metrics after each 10% chunk
                    # or if in val update on last batch
                    if ((phase == 'train') & (batch % (max(phase_size // 10, 1)) == 0)) |\
                    ((phase == 'val') & (batch == (phase_size - 1))):
                        phase_metrics = metric_collection.compute()

                        phase_metrics_dict = format_metrics_dict(
                            loss, #/ running_size, 
                            phase_metrics, 
                            phase
                        )
                        epoch_iter.set_postfix(phase_metrics_dict)
                        
                    

            # reset metric collection
            metric_collection.reset()
            
            # save the model weights if the current val monitor metric is the best so far
            if (save_dir is not None) & is_metric_better(monitor_metric, phase_metrics_dict, best_metric):
                best_metric = phase_metrics_dict[monitor_metric]
                
                print(f"saving model with best {monitor_metric} '{best_metric:.4f}'...")
                torch.save(model.state_dict(), model_save_path)

    # load best model weights and evaluate on test set
    model.load_state_dict(torch.load(model_save_path))
    id_list, pred_list, label_list = evaluate_model(model, loader_dict['test'], metric_collection, criterion)
    return id_list, pred_list, label_list

def evaluate_model(model, test_loader, metric_collection, criterion):
    running_loss = 0.0
    
    tweet_id_list = []
    pred_list = []
    label_list = []

    phase_size = len(test_loader)

    # iterate over data in current phase loader
    with tqdm(test_loader, unit="batch", total=phase_size) as epoch_iter:
        for batch, data in enumerate(epoch_iter):
            # unpack data dict
            batch_id_list = data['tweet_id_list']
            id_tensor = data['id_tensor'].to(device)
            mask_tensor = data['mask_tensor'].to(device)
            token_type_tensor = data['token_type_tensor'].to(device)
            label_tensor = data['label_tensor'].to(device)

            # zero the parameter gradients
            # optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(False):
                outputs = model(
                    id_tensor,
                    mask_tensor,
                    token_type_tensor
                )
                preds = torch.squeeze(outputs)
                loss = criterion(preds, label_tensor)

                # update running loss
                running_loss += loss.item()

                # update metric collection
                metric_collection.update(preds, label_tensor)
                
                tweet_id_list += batch_id_list
                pred_list.append(preds.detach().cpu()) #.numpy())
                label_list.append(label_tensor.detach().cpu().numpy())

    phase_metrics = metric_collection.compute()

    phase_metrics_dict = format_metrics_dict(
        loss,
        phase_metrics, 
        'test'
    )

    # print metrics
    for k, v in phase_metrics_dict.items():
        print(f"{k}: {v:.4f}")
        
    return tweet_id_list, pred_list, label_list

def is_metric_better(monitor_metric, metrics_dict, best_eval):
    """
    function to determine if the monitor metric should be maximized or minimized
    """
    curr_eval = metrics_dict.get(monitor_metric)
    if curr_eval is None:
        return False
    
    if monitor_metric.split('_')[-1] == 'loss':
        return curr_eval < best_eval
    else:
        return curr_eval > best_eval
    
def format_metrics_dict(loss, metrics_dict, set_name: str):
    out_metrics_dict = {}
    out_metrics_dict[f'{set_name}_loss'] = loss.item()

    for k, v in metrics_dict.items():
        out_metrics_dict[f'{set_name}_{k}'] = v.item()

    return out_metrics_dict

In [17]:
# define metric collection
TASK_TYPE = 'binary'
NUM_CLASSES = 2

metric_collection = MetricCollection({
    'acc': Accuracy(task=TASK_TYPE, num_classes=NUM_CLASSES),
    'auc': AUROC(task=TASK_TYPE, num_classes=NUM_CLASSES),
    'prec': Precision(task=TASK_TYPE, num_classes=NUM_CLASSES),
    'rec': Recall(task=TASK_TYPE, num_classes=NUM_CLASSES),
    'f1': F1Score(task=TASK_TYPE, num_classes=NUM_CLASSES)
})

metric_collection.to(device)

MetricCollection(
  (acc): BinaryAccuracy()
  (auc): BinaryAUROC()
  (f1): BinaryF1Score()
  (prec): BinaryPrecision()
  (rec): BinaryRecall()
)

In [18]:
# define loss and optimizer
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.000005)

In [19]:
loader_dict = {'train': train_loader, 'val': val_loader, 'test': test_loader}

train_out_tuple = train_model(
    model, 
    loader_dict, 
    metric_collection, 
    criterion, 
    optimizer, 
    save_dir="./model_test_f1_4", 
    num_epochs=30, 
    monitor_metric='val_f1'
)


Epoch 0 ----------------------------------------


100%|██████████| 6/6 [00:04<00:00,  1.32batch/s, val_loss=0.623, val_acc=0.839, val_auc=0.642, val_f1=0, val_prec=0, val_rec=0]


saving model with best val_f1 '0.0000'...

Epoch 1 ----------------------------------------


100%|██████████| 6/6 [00:04<00:00,  1.33batch/s, val_loss=0.513, val_acc=0.839, val_auc=0.69, val_f1=0, val_prec=0, val_rec=0]



Epoch 2 ----------------------------------------


...


Epoch 28 ----------------------------------------


100%|██████████| 6/6 [00:04<00:00,  1.33batch/s, val_loss=0.437, val_acc=0.927, val_auc=0.971, val_f1=0.789, val_prec=0.738, val_rec=0.849]



Epoch 29 ----------------------------------------


100%|██████████| 6/6 [00:04<00:00,  1.21batch/s]

test_loss: 0.2447
test_acc: 0.9213
test_auc: 0.9550
test_f1: 0.7759
test_prec: 0.7258
test_rec: 0.8333


In [20]:
model.load_state_dict(torch.load("/opt/localdata/Data/bea/nlp/bmi550/project/model_test_f1_2/best_model_params.pth"))

id_list, pred_list, label_list = evaluate_model(model, test_loader, metric_collection, criterion)

100%|██████████| 6/6 [00:04<00:00,  1.23batch/s]

test_loss: 0.3630
test_acc: 0.9198
test_auc: 0.9368
test_f1: 0.7759
test_prec: 0.7258
test_rec: 0.8333
